<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achat_natural_conversation_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install achatbot

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [3]:
%cd /content/achatbot


/content/achatbot


In [ ]:
!bash scripts/pypi_achatbot.sh dev


In [ ]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[fastapi_bot_server,ngrok_proxy,daily_room_audio_stream,silero_vad_analyzer,sense_voice_asr,deepgram_asr_processor,litellm_processor,google_ai,together_ai,tts_edge,queue]"


# init api key

run bot task woker with bot.json, e.g.: dummy_bot.json

- use webrtc protocol audio stream e.g.: daily,livekit
  - [daily](https://www.daily.co/) need DAILY_API_KEY
  - [livekit](https://livekit.io/) need project url LIVEKIT_URL
- asr(tts) deepgram need api key
  - [deepgram](https://developers.deepgram.com/docs/create-additional-api-keys) DEEPGRAM_API_KEY
- use openai/groq/together.ai api llm model need api key
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [groq](https://groq.com/) GROQ_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
- use image gen model:
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
  - [huggingface api](https://huggingface.co/docs/api-inference/tasks/text-to-image) HF_API_KEY




In [14]:
from google.colab import userdata
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')
DAILY_API_KEY=userdata.get('DAILY_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
DEEPGRAM_API_KEY=userdata.get('DEEPGRAM_API_KEY')


# download model

In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False


# run bot with config

In [10]:
!cat /content/daily_natural_conversation_bot.json

{
  "chat_bot_name": "DailyNaturalConversationBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "groq",
    "nlp_task_llm": "together",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "deepgram_asr_processor",
      "args": {
        "language": "zh",
        "model": "nova-2"
      }
    },
    "nlp_task_llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "google/gemma-2-27b-it",
      "language": "zh"
    },
    "llm": {
      "tag": "litellm_processor",
      "model": "gemini/gemini-1.5-flash-latest",
      "language": "zh"
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
        "voice_name": "zh-CN

In [18]:
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY DEEPGRAM_API_KEY=$DEEPGRAM_API_KEY GEMINI_API_KEY=$GOOGLE_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_natural_conversation_bot.json

{
    "chat_bot_name": "DailyNaturalConversationBot",
    "config": {
        "asr": {
            "args": {
                "language": "zh",
                "model": "nova-2"
            },
            "tag": "deepgram_asr_processor"
        },
        "llm": {
            "language": "zh",
            "model": "gemini/gemini-1.5-flash-latest",
            "tag": "litellm_processor"
        },
        "nlp_task_llm": {
            "base_url": "https://api.together.xyz/v1",
            "language": "zh",
            "model": "google/gemma-2-27b-it",
            "tag": "openai_llm_processor"
        },
        "tts": {
            "args": {
                "gender": "Male",
                "language": "zh",
                "voice_name": "zh-CN-YunjianNeural"
            },
            "tag": "tts_edge"
        },
        "vad": {
            "args": {
                "stop_secs": 0.7
            },
            "tag": "silero_vad_analyzer"
        }
    },
    "config_list": [],
    "roo

# bot task queue redis

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')


In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json